In [1]:
import pickle

def export_model(model, name):
    filename = f"{name}.plk" 
    with open(filename, 'wb') as file:
           pickle.dump(model, file)
    print(f"{filename} exported successfully")

def export_transformed_dataset(df, name):
    filename = f"{name}.pkl"
    df.to_pickle(filename)
    print(f"{filename} exported successfully")

In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
from streamlit_jupyter import StreamlitPatcher

# Enable Streamlit inside Jupyter
StreamlitPatcher().jupyter()

# Load Pretrained Model 

@st.cache_resource
def load_model(path):
    with open(path, "rb") as f:
        return pickle.load(f)

model = load_model("neural_network_model2.plk")

# Load Dataset Schema (X_train_unsampled)

X_train = pd.read_pickle("train.pkl")  # <-- Ensure you saved it with pickle
feature_info = {}

# Extract column types directly from X_train
for col in X_train.columns:
    dtype = X_train[col].dtype
    if pd.api.types.is_numeric_dtype(dtype):
        feature_info[col] = float if dtype in [np.float32, np.float64] else int
    elif pd.api.types.is_bool_dtype(dtype):
        feature_info[col] = "bool"
    else:
        feature_info[col] = "category"


# Editable Table for Input

default_data = {
    col: [0 if dtype in [int, float] else False if dtype == "bool" else ""]
    for col, dtype in feature_info.items()
}

input_df = pd.DataFrame(default_data, columns=feature_info.keys())

st.subheader("Input Transaction Features")
edited_df = st.data_editor(input_df, num_rows="fixed", key="input_table")


# Prediction

if st.button("Predict Fraud"):
    row = edited_df.iloc[0:1].copy()

    # Ensure category/boolean consistency with training schema
    for col, dtype in feature_info.items():
        if dtype == "bool":
            row[col] = row[col].astype(bool)
        elif dtype == "category":
            row[col] = row[col].astype("category")

    prediction = model.predict(row)[0]
    proba = None
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(row)[0][1]

    st.subheader("Prediction Result")
    if prediction == 1:
        st.error("This transaction is predicted as **FRAUD**.")
    else:
        st.success("This transaction is predicted as **LEGITIMATE**.")

    if proba is not None:
        st.write(f"Fraud Probability: **{proba:.2f}**")
